In [1]:
cd ../../

/Users/in-divye.singh/Documents/Projects/MIC_predictor


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from notebooks.pcm.utils import ZScalesExtractor

In [4]:
mic = pd.read_csv("data/raw/MIC_pIC50_values.csv")

In [5]:
mic

,Sequence,pIC50,MIC
0,AAQRRGRVGRNPNQVGD,7.724201,442.00000
1,HRILARIRQMMT,7.739016,435.50000
2,RNPSQVGD,7.867476,383.00000
3,RVGRNPNQVGD,7.891255,374.00000
4,AAQRRGRIGRNPSQVGD,7.934978,358.00000
...,...,...,...
707,NGAICWGPCPTAFRQIGNCGHFKVRCCKIR,15.977030,0.11515
708,CFPYITRPGTYHDWWYTRKNRQ,15.019483,0.30000
709,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,18.297579,0.01131
710,GTNLSVPNPLGFFPDHQLDPAFGANSNNPDWDFNPNKDHWPEANKVG,23.248994,0.00008


In [6]:
seq = pd.read_csv("data/raw/peptide_target_seq.csv")

In [7]:
seq = seq.merge(mic, on='Sequence')

In [8]:
seq

,Sequence,Abbreviation,Target,Target_domain,Target_seq,Target_seq_region,pIC50,MIC
0,VSTALPQWRIYSYAGDNI,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,10.596635,25.000
1,ALPQWRIYSYAGDNIVTA,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,10.596635,25.000
2,AGALMFAWLLLGLQGIFN,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,10.596635,25.000
3,MASAGMQILGVVLTLLGW,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,10.596635,25.000
4,MANSGLQLLGFSMALLGW,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,10.596635,25.000
5,MASTGLELLGMTLAVLGW,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,10.596635,25.000
6,GWIGAIVSTALPQWRIYS,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,10.747458,21.500
7,AFLGWIGAIVSTALPQWR,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,11.289782,12.500
8,FILAFLGWIGAIVSTALP,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,11.629459,8.900
9,MANAGLQLLGFILAFL,HCV,Virus entry,"HCV glycoproteins: E1, E2",YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,YQVRNSSGLYHVTNDCPNSSVVYEAADAILHTPGCVPCVREGNASR...,11.787362,7.600


In [9]:
zscale = ZScalesExtractor("data/raw/z_scales_5.csv", target_col_name="Target_seq_region")

In [10]:
zscales_features = zscale.transform(seq)

In [12]:
zscales_features

,p_z1,p_z2,p_z3,p_z4,p_z5,t_z1,t_z2,t_z3,t_z4,t_z5,...,x_z16,x_z17,x_z18,x_z19,x_z20,x_z21,x_z22,x_z23,x_z24,x_z25
0,-0.324719,0.604022,0.428253,-0.671941,0.586723,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,0.716686,-0.475155,-0.762125,-0.822596,0.836295,-0.625793,0.414894,0.665469,0.718271,-0.730233
1,-0.430982,0.477833,0.377532,-0.498596,0.706605,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,0.531798,-0.352576,-0.565514,-0.610385,0.620550,-0.753659,0.499667,0.801441,0.865032,-0.879438
2,-1.504976,-0.594266,0.325889,-0.120010,0.750372,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,0.128001,-0.084863,-0.136116,-0.146917,0.149363,-0.800340,0.530616,0.851082,0.918611,-0.933910
3,-1.280589,-1.667374,-0.121374,-0.352986,0.130028,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,0.376491,-0.249609,-0.400361,-0.432128,0.439325,-0.138687,0.091948,0.147480,0.159181,-0.161832
4,-1.020616,-0.619504,0.585948,-0.175480,0.640004,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,0.187166,-0.124089,-0.199032,-0.214824,0.218402,-0.682622,0.452571,0.725901,0.783498,-0.796546
5,-1.067569,-1.601756,0.060298,-0.656687,-0.005077,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,0.700416,-0.464368,-0.744823,-0.803921,0.817310,0.005416,-0.003590,-0.005759,-0.006216,0.006319
6,-0.853067,-0.086482,0.044621,0.069977,0.091970,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,-0.074637,0.049483,0.079369,0.085666,-0.087093,-0.098095,0.065036,0.104314,0.112591,-0.114466
7,-1.061638,-0.219738,0.072287,0.329302,0.483967,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,-0.351230,0.232861,0.373498,0.403134,-0.409847,-0.516194,0.342231,0.548921,0.592476,-0.602343
8,-1.718984,-1.030374,0.276091,-0.379334,0.784624,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,0.404595,-0.268241,-0.430246,-0.464384,0.472118,-0.836873,0.554837,0.889931,0.960543,-0.976540
9,-1.557366,-0.803360,0.186984,-0.586829,1.307206,-1.066591,0.707138,1.134213,1.224208,-1.244596,...,0.625906,-0.414969,-0.665589,-0.718400,0.730364,-1.394255,0.924375,1.482651,1.600292,-1.626944


In [ ]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
seq.shape

In [ ]:
def optimise_rf_cv(X, y, n_comp):
    # Define PLS object
    rf = RandomForestRegressor(n_estimators=n_comp)

    # Cross-validation
    y_cv = cross_val_predict(rf, X, y, cv=50)

    # Calculate scores
    r2 = r2_score(y, y_cv)
    mse = mean_squared_error(y, y_cv)
    mape = mean_absolute_percentage_error(y, y_cv)
    rpd = y.std()/np.sqrt(mse)
    
    return (y_cv, r2, mse, mape, rpd)

In [ ]:
r2s = []
mses = []
mapes = []
rpds = []
xticks = [100, 200, 250, 300, 500, 1000, 1500]#zscales_features.shape[1]-1)
for n_comp in xticks:
    y_cv, r2, mse, mape, rpd = optimise_rf_cv(zscales_features, seq['pIC50'], n_comp)
    r2s.append(r2)
    mses.append(mse)
    mapes.append(mape)
    rpds.append(rpd)

In [ ]:
def plot_metrics(vals, ylabel, objective):
    with plt.style.context('ggplot'):
        plt.plot(xticks, np.array(vals), '-v', color='blue', mfc='blue')
        if objective=='min':
            idx = np.argmin(vals)
        else:
            idx = np.argmax(vals)
        plt.plot(xticks[idx], np.array(vals)[idx], 'P', ms=10, mfc='red')

        plt.xlabel('Number of PLS components')
        plt.xticks = xticks
        plt.ylabel(ylabel)
        plt.title('PLS')

    plt.show()

In [ ]:
plot_metrics(mses, 'MSE', 'min')

In [ ]:
plot_metrics(mapes, 'MAPE', 'min')

In [ ]:
plot_metrics(r2s, 'R2', 'max')

In [ ]:
plot_metrics(rpds, 'RPD', 'max')

In [ ]:
pd.DataFrame(list(zip(mses, mapes, r2s, rpds)), columns=['MSE', 'MAPE', 'R2', 'RPD'])

In [ ]:
y_pred_mic = np.exp(-y_cv)/1e-6

In [ ]:
mean_absolute_percentage_error(seq['pIC50'], y_pred_mic)